In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import warnings
import itertools
import numpy as np 
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, log_loss

import pandas as pd
import numpy as np
import random
import pickle
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import Lasso, ElasticNet, LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.feature_selection import VarianceThreshold, RFE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=Warning)

import random
import math
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import TheilSenRegressor
from tqdm import tqdm_notebook

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

dtypes = {"crew": "int8",
          "experiment": "category",
          "time": "float32",
          "seat": "int8",
          "eeg_fp1": "float32",
          "eeg_f7": "float32",
          "eeg_f8": "float32",
          "eeg_t4": "float32",
          "eeg_t6": "float32",
          "eeg_t5": "float32",
          "eeg_t3": "float32",
          "eeg_fp2": "float32",
          "eeg_o1": "float32",
          "eeg_p3": "float32",
          "eeg_pz": "float32",
          "eeg_f3": "float32",
          "eeg_fz": "float32",
          "eeg_f4": "float32",
          "eeg_c4": "float32",
          "eeg_p4": "float32",
          "eeg_poz": "float32",
          "eeg_c3": "float32",
          "eeg_cz": "float32",
          "eeg_o2": "float32",
          "ecg": "float32",
          "r": "float32",
          "gsr": "float32",
          "event": "category",
         }

def log_loss_wrapper(y, y_pred, **kwargs):
    y = np.round(y)
    y_pred = np.round(y_pred).astype('int32')

    y_pred_onehot = np.zeros((y_pred.shape[0], 4))
    for b, i in enumerate(y_pred):
        y_pred_onehot[b, y_pred[b]] = 1

    y_onehot = np.zeros((y.shape[0], 4))
    for b, i in enumerate(y):
        y_onehot[b, y[b]] = 1
    
    return log_loss(y_onehot, y_pred_onehot, **kwargs)
    

# Custom scorer for cross validation
scorer = make_scorer(log_loss_wrapper, greater_is_better=True)

In [ ]:
train_df = pd.read_csv("../input/train.csv", dtype=dtypes)
train_df = train_df.sample(frac=1)

#test_df = pd.read_csv("../input/test.csv", dtype=dtypes)

In [ ]:
features_n = ["eeg_fp1", "eeg_f7", "eeg_f8", "eeg_t4", "eeg_t6", "eeg_t5", "eeg_t3", "eeg_fp2", "eeg_o1", "eeg_p3", "eeg_pz", "eeg_f3", "eeg_fz", "eeg_f4", "eeg_c4", "eeg_p4", "eeg_poz", "eeg_c3", "eeg_cz", "eeg_o2", "ecg", "r", "gsr"]
train_df['pilot'] = 100 * train_df['seat'] + train_df['crew']
#test_df['pilot'] = 100 * test_df['seat'] + test_df['crew']

def normalize_by_pilots(df):
    pilots = df["pilot"].unique()
    for pilot in tqdm(pilots):
        ids = df[df["pilot"] == pilot].index
        scaler = MinMaxScaler()
        df.loc[ids, features_n] = scaler.fit_transform(df.loc[ids, features_n])
    return df

train_df = normalize_by_pilots(train_df)


In [ ]:
dic = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
dic1 = {'CA': 2, 'SS': 1, 'DA': 3, 'LOFT': 0}
    
#train_result_df = train_df["event"]
try:
    train_result_df = train_df.pop("event")
except:
    pass

try:
    train_df[["experiment"]] = train_df["experiment"].apply(lambda x: dic1[x])
    train_result_df = train_result_df.apply(lambda x: dic[x])
except:
    pass


train = train_df.values
train_result = np.array(train_result_df.values)

train_result_onehot = np.zeros((train_result.shape[0], 4))
for b, i in enumerate(train_result):
    train_result_onehot[b, train_result[b]] = 1
    
#test_df = normalize_by_pilots(test_df)

In [ ]:
def randomizeBits(size):
    return np.random.randint(2, size=size)

def getCardinality(threshold, accumulation):
    for i in range(0, len(accumulation)):
        if (accumulation[i] > threshold):
            return i;
    return len(accumulation) - 1;

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

lasso = make_pipeline(RobustScaler(), Lasso(alpha =10, random_state=1, max_iter = 14000))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=1, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
forest_model = RandomForestRegressor()
svr = svm.SVR()
Linearsvr = svm.LinearSVR()
Nusvr = svm.NuSVR()
KNN = KNeighborsRegressor(n_neighbors=5)
theil = TheilSenRegressor(random_state=0)
logistic = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial')
gnb = GaussianNB()
models = [lasso, ENet, KRR, GBoost, svr, Linearsvr, Nusvr, KNN, logistic, gnb, forest_model]
#models = [lasso, ENet, KRR, GBoost, model_xgb, model_lgb, forest_model]
#models = [lasso, ENet, KRR, GBoost, model_xgb, forest_model]

class MetaModel(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)

        kfold = KFold(n_splits=n_folds, shuffle=True, random_state=156)

        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model

        #placeholder
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))

        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred

        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    #Do the predictions of all base models on the test data and use the averaged predictions as
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

n_folds = 2
def cv_model(model):
    """Perform 10 fold cross validation of a model"""
    
    random_set = np.random.choice(train.shape[0], 2000, replace=False)
    cv_scores = cross_val_score(model, train[random_set], train_result[random_set], cv = n_folds, scoring=scorer, n_jobs = -1)
    #cv_scores = cross_val_score(model, train[:2000000], train_result[0:2000000], cv = n_folds, scoring=scorer, n_jobs = -1)
    #cv_scores = cross_val_score(model, train, train_result, cv = n_folds, scoring=scorer, n_jobs = -1)
    print(f'5 Fold CV Score: {round(cv_scores.mean(), 5)} with std: {round(cv_scores.std(), 5)}')
    
    return cv_scores

In [ ]:
class Particle:
    def __init__(self, models):
        self.num_dimensions = len(models)
        self.meta_index = np.zeros((self.num_dimensions))
        self.meta_index[0] = np.random.randint(self.num_dimensions-1)
        #if(self.meta_index[0] in [1, 2, 5]):
        #    self.meta_index[0] = 0
        self.position = np.vstack((randomizeBits(len(models)), self.meta_index))        
        self.velocity = np.zeros((2, self.num_dimensions))
        self.pbest_pos = []
        self.pbest_error = -1
        self.curr_error = -1
        self.models = models

        for i in range(0, self.num_dimensions):
            self.velocity[0, i] = random.uniform(-1, 1)

    def calculate_cost(self):
        break_boolean = False
        for i in range(0, self.num_dimensions):
            if(self.position[0][i] != 0):
                break_boolean = True
        if(break_boolean == False):
            return

        instance_meta_model = models[int(self.position[1][0])]
        instance_base_models = []

        for i in range(0, self.num_dimensions):
            if(self.position[0][i] == 1):
                instance_base_models.append(models[i])

        meta_model = MetaModel(base_models = instance_base_models, meta_model = instance_meta_model)
        #meta_model.fit(train, train_result.reshape(-1))

        score = cv_model(meta_model)
        self.curr_error = score.mean()

        print("Error: " + str(self.curr_error))
        print("Position: " + str(self.position))

        if(self.curr_error < self.pbest_error or self.pbest_error == -1):
            self.pbest_pos = self.position
            self.pbest_error = self.curr_error

    def updateVelocity(self, inertia, cog_const, soc_const, gbest_pos):
        for i in range(0, self.num_dimensions):
            r1=random.random()
            r2=random.random()

            card_count_soc = card_count_cog = card_count_self = 0

            # Get Encoding of Cardinality
            for j in self.pbest_pos[0]:
                card_count_cog += int(j)
            for j in gbest_pos[0]:
                card_count_soc += int(j)
            for j in self.position[0]:
                card_count_self += int(j)

            # Encode Cardinality
            L_cog = np.zeros((2, self.num_dimensions))
            L_soc = np.zeros((2, self.num_dimensions))
            L_self = np.zeros((2, self.num_dimensions))
            #print(card_count_cog, card_count_soc, card_count_self)
            L_cog[0, card_count_cog - 1] = 1
            L_soc[0, card_count_soc - 1] = 1
            L_self[0, card_count_self - 1] = 1

            # Include Selection Likelihood
            #print(self.pbest_pos)
            #print(self.position)
            for a in range(0, self.num_dimensions):
                if(self.pbest_pos[0][a] == 1 and self.position[0][a] == 0):
                    L_cog[1, a] = 1;
                if(gbest_pos[0][a] == 1 and self.position[0][a] == 0):
                    L_soc[1, a] = 1;

            vel_cognitive = cog_const * r1 * L_cog
            vel_social = soc_const * r2 * L_soc
            vel_self = 0.2 * L_self
            self.velocity = inertia * self.velocity + vel_cognitive + vel_social + vel_self

    def updatePosition(self, meta_choice_array):
        # Create Accumulation Matrix
        accumulation = np.zeros((self.num_dimensions))
        for i in range(0, self.num_dimensions):
            if (i == 0):
                accumulation[i] = self.velocity[0, i]
            else:
                accumulation[i] = self.velocity[0, i] + accumulation[i-1]

        # Get Random Threshold
        random_thresh = random.random() * accumulation[-1]

        # Calculate Cardinality
        cardinality = getCardinality(random_thresh, accumulation)

        # Fill in updated position with maximum selection likelihoods
        temp_selection = np.copy(self.velocity[1, :])
        print("Velocity: " + str(self.velocity))
        updated_position = np.zeros((2, self.num_dimensions))
        for i in range(cardinality):
            max_index = np.argmax(temp_selection)
            #print("Max Index is: " + str(max_index))
            updated_position[0][max_index] = 1
            temp_selection[max_index] = float('-inf')
            
        print(meta_choice_array)
            
        x = np.random.choice(range(0, self.num_dimensions), 1, p= meta_choice_array)
        print("Initial: " + str(x))
        print("Later: " + str(x))
        
        self.position = updated_position
        if(random.randint(0,1) == 1):
            self.position[1][0] = x

class PSO:
    def __init__(self, num_particles, epoch, continue_flag = False, pso = b'a'):
        self.num_dimensions = len(models)
        self.gbest_pos = []
        self.gbest_error = -1
        self.inertia = 0.8
        self.cog_const = 1
        self.soc_const = 2
        self.epoch = epoch
        self.num_particles = num_particles
        self.epsilon = 1e6
        self.meta_weights = np.full((self.num_dimensions), self.epsilon)
        self.meta_size = np.zeros((self.num_dimensions))
        self.real_meta_array = np.full((self.num_dimensions), self.epsilon)
        self.global_history = []

        if(continue_flag):
            self.swarm = pickle.loads(pso)
        else:
            self.swarm = []
            for i in range(0, num_particles):
                self.swarm.append(Particle(models))

    def run(self):
        print("")
        for timestep in range(self.epoch):
            print("Timestep: %d" % timestep)
            
            # Update Meta Array
            for i in range(0, self.num_dimensions):
                if(self.meta_size[i] != 0):
                    self.real_meta_array[i] = self.meta_weights[i] / self.meta_size[i]
                if(self.meta_size[i] == 0):
                    self.real_meta_array[i] = self.epsilon
            print("Weights: " + str(self.meta_weights))
            print("Size: " + str(self.meta_size))
            print("Meta Array: " + str(self.real_meta_array))
            self.real_meta_array = softmax(1 / self.real_meta_array)
            print("After Meta Array: " + str(self.real_meta_array))
            
            # Reset Weights and Size arrays
            self.meta_weights = np.full((self.num_dimensions), self.epsilon)
            self.meta_size = np.zeros((self.num_dimensions))
            
            # Loop Each Particle
            for i in range(0, self.num_particles):
                print(self.swarm[i].position)
                # Calculate Cost
                self.swarm[i].calculate_cost()
                
                # Add weights and increment index
                meta_ind = int(self.swarm[i].position[1][0])
                if(self.meta_size[meta_ind] == 0):
                    self.meta_weights[meta_ind] = 0
                self.meta_weights[meta_ind] += self.swarm[i].curr_error
                self.meta_size[meta_ind] += 1

                if self.swarm[i].curr_error < self.gbest_error or self.gbest_error == -1:
                    self.gbest_pos = list(self.swarm[i].position)
                    self.gbest_error = float(self.swarm[i].curr_error)

                self.swarm[i].updateVelocity(self.inertia, self.cog_const, self.soc_const, self.gbest_pos)
                self.swarm[i].updatePosition(self.real_meta_array)
                print("Pickled: " + str(timestep) + " " + str(i))
                print(pickle.dumps(self))
                
            self.global_history.append(self.gbest_error)

        # Revisit Later
        print("---------------------------------")
        print("Final:")
        print("Gbest Position: " + str(self.gbest_pos))
        print("Gbest Error: " + str(self.gbest_error))
        


In [ ]:
pso = PSO(num_particles = 50, epoch = 5)
#1
#pso = pickle.loads(b'\x80\x03c__main__\nPSO\nq\x00)\x81q\x01}q\x02(X\x0e\x00\x00\x00num_dimensionsq\x03K\x0bX\t\x00\x00\x00gbest_posq\x04]q\x05(cnumpy.core._multiarray_umath\n_reconstruct\nq\x06cnumpy\nndarray\nq\x07K\x00\x85q\x08C\x01bq\t\x87q\nRq\x0b(K\x01K\x0b\x85q\x0ccnumpy\ndtype\nq\rX\x02\x00\x00\x00f8q\x0eK\x00K\x01\x87q\x0fRq\x10(K\x03X\x01\x00\x00\x00<q\x11NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x12b\x89CX\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00q\x13tq\x14bh\x06h\x07K\x00\x85q\x15h\t\x87q\x16Rq\x17(K\x01K\x0b\x85q\x18h\x10\x89CX\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00q\x19tq\x1abeX\x0b\x00\x00\x00gbest_errorq\x1bG@\x02\xed`8\x8c\xe1\nX\x07\x00\x00\x00inertiaq\x1cG?\xe9\x99\x99\x99\x99\x99\x9aX\t\x00\x00\x00cog_constq\x1dK\x01X\t\x00\x00\x00soc_constq\x1eK\x02X\x05\x00\x00\x00epochq\x1fK\x05X\r\x00\x00\x00num_particlesq K\nX\x07\x00\x00\x00epsilonq!GA.\x84\x80\x00\x00\x00\x00X\x0c\x00\x00\x00meta_weightsq"h\x06h\x07K\x00\x85q#h\t\x87q$Rq%(K\x01K\x0b\x85q&h\rX\x02\x00\x00\x00f8q\'K\x00K\x01\x87q(Rq)(K\x03h\x11NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq*b\x89CXI\x90\xda\'\xd18i@\x00\x00\x00\x00\x80\x84.AFG\x9d\x86\xa1\xab\x0e@\x00\x00\x00\x00\x80\x84.A\x00\x00\x00\x00\x80\x84.AX\x19\xe5Inz2@\x8a\xcc\xeeHQ\x08\x04@\xd8-\x839}_!@\x00\x00\x00\x00\x80\x84.A\x00\x00\x00\x00\x80\x84.A\x00\x00\x00\x00\x80\x84.Aq+tq,bX\t\x00\x00\x00meta_sizeq-h\x06h\x07K\x00\x85q.h\t\x87q/Rq0(K\x01K\x0b\x85q1h)\x89CX\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00q2tq3bX\x0f\x00\x00\x00real_meta_arrayq4h\x06h\x07K\x00\x85q5h\t\x87q6Rq7(K\x01K\x0b\x85q8h)\x89CX\x8fK\xe1\xcb\x96\xf9\xb5?\xfa+\nm\x14U\xb5?\xfa+\nm\x14U\xb5?\xfa+\nm\x14U\xb5?\x8e\x7f\xfa\x85\xcb\x0c\xbe?\xfa+\nm\x14U\xb5?\xfa+\nm\x14U\xb5?\xfa+\nm\x14U\xb5?\xfa+\nm\x14U\xb5?\xca\xd4\t/\x07\xee\xbd?H,\xd3\x83\x07\xb8\xb8?q9tq:bX\x0e\x00\x00\x00global_historyq;]q<(G@\x03Wz\x9e\xb1\x99ZG@\x03Wz\x9e\xb1\x99ZG@\x03Wz\x9e\xb1\x99ZG@\x03Wz\x9e\xb1\x99ZG@\x02\xed`8\x8c\xe1\nG@\x02\xed`8\x8c\xe1\nG@\x02\xed`8\x8c\xe1\nG@\x02\xed`8\x8c\xe1\neX\x05\x00\x00\x00swarmq=]q>(c__main__\nParticle\nq?)\x81q@}qA(h\x03K\x0bX\n\x00\x00\x00meta_indexqBh\x06h\x07K\x00\x85qCh\t\x87qDRqE(K\x01K\x0b\x85qFh\x10\x89CX\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00qGtqHbX\x08\x00\x00\x00positionqIh\x06h\x07K\x00\x85qJh\t\x87qKRqL(K\x01K\x02K\x0b\x86qMh)\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00qNtqObX\x08\x00\x00\x00velocityqPh\x06h\x07K\x00\x85qQh\t\x87qRRqS(K\x01K\x02K\x0b\x86qTh)\x89C\xb0M7\x18\xba\']\xb3\xbd,Cz9D\xe2\xb3\xbd\xa7\xfb\xe6\x02\x8c\x96\xab\xbdp\xd9\xf4\xab\x01r\n@\xe4\x96i\x14h\xee\xc5?\xd6\x9b\xc1\x03_?\x17@\xa0\'\x01L\xd7H\xfc>:\xb2\xa5\xeb\xca\x8d\x9b=\xc4\xa6\x8f\xa3\x00\xf5\xa5\xbdd\xf0]\xaa!\xb7\xb4\xbd\xde\xae\xbf\x88\xbd\xc3\xb5=\xae!G\x068\xc0\x15@\xae!G\x068\xc0\x15@\xf7\x10\x7f\xabR\x10\xdf?\xf7\x10\x7f\xabR\x10\xdf?\xf7\x10\x7f\xabR\x10\xdf?\x00\x00\x00\x00\x00\x00\x00\x00\xc5\x0f7\x0f\x98O\xaa?\xcf\x97\xa2hc\xb4\xdc>\x00\xcd\xf7\xfb H\xb5>\x00\x00\x00\x00\x00\x00\x00\x00\xb7@\x94t\r_\x94?qUtqVbX\t\x00\x00\x00pbest_posqWh\x06h\x07K\x00\x85qXh\t\x87qYRqZ(K\x01K\x02K\x0b\x86q[h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00q\\tq]bX\x0b\x00\x00\x00pbest_errorq^cnumpy.core._multiarray_umath\nscalar\nq_h)C\x08\x8a\xcc\xeeHQ\x08\x04@q`\x86qaRqbX\n\x00\x00\x00curr_errorqchbX\x06\x00\x00\x00modelsqd]qe(csklearn.pipeline\nPipeline\nqf)\x81qg}qh(X\x05\x00\x00\x00stepsqi]qj(X\x0c\x00\x00\x00robustscalerqkcsklearn.preprocessing.data\nRobustScaler\nql)\x81qm}qn(X\x0e\x00\x00\x00with_centeringqo\x88X\x0c\x00\x00\x00with_scalingqp\x88X\x0e\x00\x00\x00quantile_rangeqqG@9\x00\x00\x00\x00\x00\x00G@R\xc0\x00\x00\x00\x00\x00\x86qrX\x04\x00\x00\x00copyqs\x88X\x10\x00\x00\x00_sklearn_versionqtX\x06\x00\x00\x000.20.2quub\x86qvX\x05\x00\x00\x00lassoqwcsklearn.linear_model.coordinate_descent\nLasso\nqx)\x81qy}qz(X\x05\x00\x00\x00alphaq{K\nX\x08\x00\x00\x00l1_ratioq|G?\xf0\x00\x00\x00\x00\x00\x00X\r\x00\x00\x00fit_interceptq}\x88X\t\x00\x00\x00normalizeq~\x89X\n\x00\x00\x00precomputeq\x7f\x89X\x08\x00\x00\x00max_iterq\x80M\xb06X\x06\x00\x00\x00copy_Xq\x81\x88X\x03\x00\x00\x00tolq\x82G?\x1a6\xe2\xeb\x1cC-X\n\x00\x00\x00warm_startq\x83\x89X\x08\x00\x00\x00positiveq\x84\x89X\x0c\x00\x00\x00random_stateq\x85K\x01X\t\x00\x00\x00selectionq\x86X\x06\x00\x00\x00cyclicq\x87hthuub\x86q\x88eX\x06\x00\x00\x00memoryq\x89Nhthuubhf)\x81q\x8a}q\x8b(hi]q\x8c(X\x0c\x00\x00\x00robustscalerq\x8dhl)\x81q\x8e}q\x8f(ho\x88hp\x88hqhrhs\x88hthuub\x86q\x90X\n\x00\x00\x00elasticnetq\x91csklearn.linear_model.coordinate_descent\nElasticNet\nq\x92)\x81q\x93}q\x94(h{K\x01h|G?\xec\xcc\xcc\xcc\xcc\xcc\xcdh}\x88h~\x89h\x7f\x89h\x80M\xe8\x03h\x81\x88h\x82G?\x1a6\xe2\xeb\x1cC-h\x83\x89h\x84\x89h\x85K\x03h\x86h\x87hthuub\x86q\x95eh\x89Nhthuubcsklearn.kernel_ridge\nKernelRidge\nq\x96)\x81q\x97}q\x98(h{G?\xe3333333X\x06\x00\x00\x00kernelq\x99X\n\x00\x00\x00polynomialq\x9aX\x05\x00\x00\x00gammaq\x9bNX\x06\x00\x00\x00degreeq\x9cK\x02X\x05\x00\x00\x00coef0q\x9dG@\x04\x00\x00\x00\x00\x00\x00X\r\x00\x00\x00kernel_paramsq\x9eNhthuubcsklearn.ensemble.gradient_boosting\nGradientBoostingRegressor\nq\x9f)\x81q\xa0}q\xa1(X\x0c\x00\x00\x00n_estimatorsq\xa2M\xb8\x0bX\r\x00\x00\x00learning_rateq\xa3G?\xa9\x99\x99\x99\x99\x99\x9aX\x04\x00\x00\x00lossq\xa4X\x05\x00\x00\x00huberq\xa5X\t\x00\x00\x00criterionq\xa6X\x0c\x00\x00\x00friedman_mseq\xa7X\x11\x00\x00\x00min_samples_splitq\xa8K\nX\x10\x00\x00\x00min_samples_leafq\xa9K\x0fX\x18\x00\x00\x00min_weight_fraction_leafq\xaaG\x00\x00\x00\x00\x00\x00\x00\x00X\t\x00\x00\x00subsampleq\xabG?\xf0\x00\x00\x00\x00\x00\x00X\x0c\x00\x00\x00max_featuresq\xacX\x04\x00\x00\x00sqrtq\xadX\t\x00\x00\x00max_depthq\xaeK\x04X\x15\x00\x00\x00min_impurity_decreaseq\xafG\x00\x00\x00\x00\x00\x00\x00\x00X\x12\x00\x00\x00min_impurity_splitq\xb0NX\x04\x00\x00\x00initq\xb1Nh\x85K\x05h{G?\xec\xcc\xcc\xcc\xcc\xcc\xcdX\x07\x00\x00\x00verboseq\xb2K\x00X\x0e\x00\x00\x00max_leaf_nodesq\xb3Nh\x83\x89X\x07\x00\x00\x00presortq\xb4X\x04\x00\x00\x00autoq\xb5X\x13\x00\x00\x00validation_fractionq\xb6G?\xb9\x99\x99\x99\x99\x99\x9aX\x10\x00\x00\x00n_iter_no_changeq\xb7Nh\x82G?\x1a6\xe2\xeb\x1cC-hthuubcsklearn.svm.classes\nSVR\nq\xb8)\x81q\xb9}q\xba(h\x99X\x03\x00\x00\x00rbfq\xbbh\x9cK\x03h\x9bX\x0f\x00\x00\x00auto_deprecatedq\xbch\x9dG\x00\x00\x00\x00\x00\x00\x00\x00h\x82G?PbM\xd2\xf1\xa9\xfcX\x01\x00\x00\x00Cq\xbdG?\xf0\x00\x00\x00\x00\x00\x00X\x02\x00\x00\x00nuq\xbeG\x00\x00\x00\x00\x00\x00\x00\x00h!G?\xb9\x99\x99\x99\x99\x99\x9aX\t\x00\x00\x00shrinkingq\xbf\x88X\x0b\x00\x00\x00probabilityq\xc0\x89X\n\x00\x00\x00cache_sizeq\xc1K\xc8X\x0c\x00\x00\x00class_weightq\xc2Nh\xb2\x89h\x80J\xff\xff\xff\xffh\x85Nhthuubcsklearn.svm.classes\nLinearSVR\nq\xc3)\x81q\xc4}q\xc5(h\x82G?\x1a6\xe2\xeb\x1cC-h\xbdG?\xf0\x00\x00\x00\x00\x00\x00h!G\x00\x00\x00\x00\x00\x00\x00\x00h}\x88X\x11\x00\x00\x00intercept_scalingq\xc6G?\xf0\x00\x00\x00\x00\x00\x00h\xb2K\x00h\x85Nh\x80M\xe8\x03X\x04\x00\x00\x00dualq\xc7\x88h\xa4X\x13\x00\x00\x00epsilon_insensitiveq\xc8hthuubcsklearn.svm.classes\nNuSVR\nq\xc9)\x81q\xca}q\xcb(h\x99h\xbbh\x9cK\x03h\x9bh\xbch\x9dG\x00\x00\x00\x00\x00\x00\x00\x00h\x82G?PbM\xd2\xf1\xa9\xfch\xbdG?\xf0\x00\x00\x00\x00\x00\x00h\xbeG?\xe0\x00\x00\x00\x00\x00\x00h!G\x00\x00\x00\x00\x00\x00\x00\x00h\xbf\x88h\xc0\x89h\xc1K\xc8h\xc2Nh\xb2\x89h\x80J\xff\xff\xff\xffh\x85Nhthuubcsklearn.neighbors.regression\nKNeighborsRegressor\nq\xcc)\x81q\xcd}q\xce(X\x0b\x00\x00\x00n_neighborsq\xcfK\x05X\x06\x00\x00\x00radiusq\xd0NX\t\x00\x00\x00algorithmq\xd1h\xb5X\t\x00\x00\x00leaf_sizeq\xd2K\x1eX\x06\x00\x00\x00metricq\xd3X\t\x00\x00\x00minkowskiq\xd4X\r\x00\x00\x00metric_paramsq\xd5NX\x01\x00\x00\x00pq\xd6K\x02X\x06\x00\x00\x00n_jobsq\xd7NX\x07\x00\x00\x00weightsq\xd8X\x07\x00\x00\x00uniformq\xd9hthuubcsklearn.linear_model.logistic\nLogisticRegression\nq\xda)\x81q\xdb}q\xdc(X\x07\x00\x00\x00penaltyq\xddX\x02\x00\x00\x00l2q\xdeh\xc7\x89h\x82G?\x1a6\xe2\xeb\x1cC-h\xbdG?\xf0\x00\x00\x00\x00\x00\x00h}\x88h\xc6K\x01h\xc2Nh\x85K\x00X\x06\x00\x00\x00solverq\xdfX\x05\x00\x00\x00lbfgsq\xe0h\x80KdX\x0b\x00\x00\x00multi_classq\xe1X\x0b\x00\x00\x00multinomialq\xe2h\xb2K\x00h\x83\x89h\xd7Nhthuubcsklearn.naive_bayes\nGaussianNB\nq\xe3)\x81q\xe4}q\xe5(X\x06\x00\x00\x00priorsq\xe6NX\r\x00\x00\x00var_smoothingq\xe7G>\x11.\x0b\xe8&\xd6\x95hthuubcsklearn.ensemble.forest\nRandomForestRegressor\nq\xe8)\x81q\xe9}q\xea(X\x0e\x00\x00\x00base_estimatorq\xebcsklearn.tree.tree\nDecisionTreeRegressor\nq\xec)\x81q\xed}q\xee(h\xa6X\x03\x00\x00\x00mseq\xefX\x08\x00\x00\x00splitterq\xf0X\x04\x00\x00\x00bestq\xf1h\xaeNh\xa8K\x02h\xa9K\x01h\xaaG\x00\x00\x00\x00\x00\x00\x00\x00h\xacNh\x85Nh\xb3Nh\xafG\x00\x00\x00\x00\x00\x00\x00\x00h\xb0Nh\xc2Nh\xb4\x89hthuubh\xa2X\x04\x00\x00\x00warnq\xf2X\x10\x00\x00\x00estimator_paramsq\xf3(X\t\x00\x00\x00criterionq\xf4X\t\x00\x00\x00max_depthq\xf5X\x11\x00\x00\x00min_samples_splitq\xf6X\x10\x00\x00\x00min_samples_leafq\xf7X\x18\x00\x00\x00min_weight_fraction_leafq\xf8X\x0c\x00\x00\x00max_featuresq\xf9X\x0e\x00\x00\x00max_leaf_nodesq\xfaX\x15\x00\x00\x00min_impurity_decreaseq\xfbX\x12\x00\x00\x00min_impurity_splitq\xfcX\x0c\x00\x00\x00random_stateq\xfdtq\xfeX\t\x00\x00\x00bootstrapq\xff\x88X\t\x00\x00\x00oob_scorer\x00\x01\x00\x00\x89h\xd7Nh\x85Nh\xb2K\x00h\x83\x89h\xc2Nh\xa6h\xefh\xaeNh\xa8K\x02h\xa9K\x01h\xaaG\x00\x00\x00\x00\x00\x00\x00\x00h\xach\xb5h\xb3Nh\xafG\x00\x00\x00\x00\x00\x00\x00\x00h\xb0Nhthuubeubh?)\x81r\x01\x01\x00\x00}r\x02\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85r\x03\x01\x00\x00h\t\x87r\x04\x01\x00\x00Rr\x05\x01\x00\x00(K\x01K\x0b\x85r\x06\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x07\x01\x00\x00tr\x08\x01\x00\x00bhIh\x06h\x07K\x00\x85r\t\x01\x00\x00h\t\x87r\n\x01\x00\x00Rr\x0b\x01\x00\x00(K\x01K\x02K\x0b\x86r\x0c\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\r\x01\x00\x00tr\x0e\x01\x00\x00bhPh\x06h\x07K\x00\x85r\x0f\x01\x00\x00h\t\x87r\x10\x01\x00\x00Rr\x11\x01\x00\x00(K\x01K\x02K\x0b\x86r\x12\x01\x00\x00h)\x89C\xb0\xb1\xe7,9h\xdfv=m\xdb\xf9\x1d\xa8\x89\x9d=B\xa2~\x94O\x16\x85\xbd\xc4\x9a\xc9\xb0\x87X\xb1\xbd>\x08\x13\xf0\xf8\xff\xef?\x89\x11\x1fdt\xa6!@_S\x97\x8a\x95\xfc\x00?\xf0\xfc\xc0\xb4R\x06\xa3\xbd\x08\xaf\xc2&\xa2\x9b\xa7=\x94}+-\x9c\xc0\xaf=h\x1f\xb6\xdd\xe7\x84\x97=\x8cW\xa1\xb5|\xfb\xdb?]\xee\xe8\x04w\xd9\xad?\x9b\xff\xfcE\xa8\xc4 @Kd\x92-\x83\xcc\x16@\xef\x89Y;4\xac\x16@\x00\x00\x00\x00\x00\x00\x00\x00\x9a\xd4\xa5\xec\xbe_\xd3?y5T<\r\xb4\xc1?\xb5{\x0e\xbcJy\x05@\x00\x00\x00\x00\x00\x00\x00\x00\xa9\xde\xcc\xac\x1b\xb4\xc1?r\x13\x01\x00\x00tr\x14\x01\x00\x00bhWh\x06h\x07K\x00\x85r\x15\x01\x00\x00h\t\x87r\x16\x01\x00\x00Rr\x17\x01\x00\x00(K\x01K\x02K\x0b\x86r\x18\x01\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x19\x01\x00\x00tr\x1a\x01\x00\x00bh^h_h)C\x08\xe6\xe0"2\xf4I\r@r\x1b\x01\x00\x00\x86r\x1c\x01\x00\x00Rr\x1d\x01\x00\x00hch_h)C\x08\xa4\x1ay\xa4\xe9\xe5@@r\x1e\x01\x00\x00\x86r\x1f\x01\x00\x00Rr \x01\x00\x00hdheubh?)\x81r!\x01\x00\x00}r"\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85r#\x01\x00\x00h\t\x87r$\x01\x00\x00Rr%\x01\x00\x00(K\x01K\x0b\x85r&\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\'\x01\x00\x00tr(\x01\x00\x00bhIh\x06h\x07K\x00\x85r)\x01\x00\x00h\t\x87r*\x01\x00\x00Rr+\x01\x00\x00(K\x01K\x02K\x0b\x86r,\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r-\x01\x00\x00tr.\x01\x00\x00bhPh\x06h\x07K\x00\x85r/\x01\x00\x00h\t\x87r0\x01\x00\x00Rr1\x01\x00\x00(K\x01K\x02K\x0b\x86r2\x01\x00\x00h)\x89C\xb08\x1cF\xc8\xf6\xba\xb0=\x8by\xf3!\xb2^\xa6=tG\xc9l\xea\xc6\xa9\xbd,\xa6c?-)\x8e=\x18\'\xf5\xd1\x8d\xff\xef?\\\x1c\x0c\xc3\x927\x0c@*\xea\xe1Q\t\xf8\x00@\xc7\xa9\x1b\xc1\xf3\xa6\xb4\xbd\x85C+\xa6p\xfe\x9d=g\xda\x1cB%\x16\xac=H\xe1\xb5z\xbc\x08\xa4=\x9d\xecc\xdb\xd0\x84m?\xbb\x192\xfa@p\x13@|\x16\xa1\\\xcfu\xe7?\x88\x15\xd0\xd7\x89u\xe7?$\xcb\xec_\xc1\x89\xaa?\x00\x00\x00\x00\x00\x00\x00\x00\xa8\xe2%\x90\xbb\x89\xaa?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00,\xb1\x97\x81\xd7\xb3\xfd?r3\x01\x00\x00tr4\x01\x00\x00bhWh\x06h\x07K\x00\x85r5\x01\x00\x00h\t\x87r6\x01\x00\x00Rr7\x01\x00\x00(K\x01K\x02K\x0b\x86r8\x01\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r9\x01\x00\x00tr:\x01\x00\x00bh^h_h)C\x08Z\x99\xb1\x9ezW\x03@r;\x01\x00\x00\x86r<\x01\x00\x00Rr=\x01\x00\x00hch_h)C\x08FG\x9d\x86\xa1\xab\x0e@r>\x01\x00\x00\x86r?\x01\x00\x00Rr@\x01\x00\x00hdheubh?)\x81rA\x01\x00\x00}rB\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85rC\x01\x00\x00h\t\x87rD\x01\x00\x00RrE\x01\x00\x00(K\x01K\x0b\x85rF\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rG\x01\x00\x00trH\x01\x00\x00bhIh\x06h\x07K\x00\x85rI\x01\x00\x00h\t\x87rJ\x01\x00\x00RrK\x01\x00\x00(K\x01K\x02K\x0b\x86rL\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rM\x01\x00\x00trN\x01\x00\x00bhPh\x06h\x07K\x00\x85rO\x01\x00\x00h\t\x87rP\x01\x00\x00RrQ\x01\x00\x00(K\x01K\x02K\x0b\x86rR\x01\x00\x00h)\x89C\xb0MhdC$\x8e\xb3=\xfa\x06e\x17\t\xb6\xb7=\x00%V\xddN;g=\\\x01g\xe0\xed\x19\xb4?*A}\x1eY|\xed?\xab(.\x06\xc2U @g\xe11\x9a\xe4U\xf8>\xff\xb8\xe5&\x01\xb2\x99=\xe5;\x8cl\x860\x9b\xbd\xe4&+~\xed\xbd\x86=\x9d\x04)\xa1\x18\xf3\xae=\x82\x12\xda\xa4\x7f\x98\xab?\x17\xce\xb7\x18\x81\x15\x1e@\\_\xe8C*\xa5\x14@\x07\x13\xa0\xe6\x08\xf6\xe2?C\xa7\xb7A\x88\xf2\xe2?\x00\x00\x00\x00\x00\x00\x00\x00C\xa7\xb7A\x88\xf2\xe2?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xfa\xc0n\x8aj\xdc\x91?rS\x01\x00\x00trT\x01\x00\x00bhWh\x06h\x07K\x00\x85rU\x01\x00\x00h\t\x87rV\x01\x00\x00RrW\x01\x00\x00(K\x01K\x02K\x0b\x86rX\x01\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x10@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rY\x01\x00\x00trZ\x01\x00\x00bh^h_h)C\x08\xae\xff]FY-\x08@r[\x01\x00\x00\x86r\\\x01\x00\x00Rr]\x01\x00\x00hch_h)C\x08\xd9\xcd\xe9\xf9\xb3\xb9@@r^\x01\x00\x00\x86r_\x01\x00\x00Rr`\x01\x00\x00hdheubh?)\x81ra\x01\x00\x00}rb\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85rc\x01\x00\x00h\t\x87rd\x01\x00\x00Rre\x01\x00\x00(K\x01K\x0b\x85rf\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rg\x01\x00\x00trh\x01\x00\x00bhIh\x06h\x07K\x00\x85ri\x01\x00\x00h\t\x87rj\x01\x00\x00Rrk\x01\x00\x00(K\x01K\x02K\x0b\x86rl\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00rm\x01\x00\x00trn\x01\x00\x00bhPh\x06h\x07K\x00\x85ro\x01\x00\x00h\t\x87rp\x01\x00\x00Rrq\x01\x00\x00(K\x01K\x02K\x0b\x86rr\x01\x00\x00h)\x89C\xb0P\xc6\xa2\x00~\xef`>8}\xcd\x95tK\'>\x90\xb5-\x96\x0e\x02\xef>*\x15s\xfe\xb4`\xad=\\\xa9\xf4\xd1\x8d\xff\xef?{;\xf7\x1cP[\x19@>\xe8\xd1\x94\xb4\x97\xf5>\'L\xf6\x8dM\x1a\xaf=l\xb5\x9c\x9c\x87o\xae=\xdd\xb5\x08\xfbR\xf2\x9f=wq"\x9e\x1a\xa5\x98>\xcdBu"\xd2\xf8\xdf?\x01\xcb\x93\xc1\xe4|\x16@\xa6C\x86\x9cN\x83\xa7?A\xfb\xbd\x8f\xe4|\x16@O\xf3\x18\x184\x98\xe5?\x00\x00\x00\x00\x00\x00\x00\x00\x88\xc4j,\xd2\xf8\xdf?s0\xc3\xc3\xcb\xe5\x92?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x1f\xd2\xce[\x00\x1b\x01@rs\x01\x00\x00trt\x01\x00\x00bhWh\x06h\x07K\x00\x85ru\x01\x00\x00h\t\x87rv\x01\x00\x00Rrw\x01\x00\x00(K\x01K\x02K\x0b\x86rx\x01\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00ry\x01\x00\x00trz\x01\x00\x00bh^h_h)C\x08\xbc\xff+\xf3\'\xb9\x04@r{\x01\x00\x00\x86r|\x01\x00\x00Rr}\x01\x00\x00hch_h)C\x08X\x19\xe5Inz2@r~\x01\x00\x00\x86r\x7f\x01\x00\x00Rr\x80\x01\x00\x00hdheubh?)\x81r\x81\x01\x00\x00}r\x82\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85r\x83\x01\x00\x00h\t\x87r\x84\x01\x00\x00Rr\x85\x01\x00\x00(K\x01K\x0b\x85r\x86\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x87\x01\x00\x00tr\x88\x01\x00\x00bhIh\x06h\x07K\x00\x85r\x89\x01\x00\x00h\t\x87r\x8a\x01\x00\x00Rr\x8b\x01\x00\x00(K\x01K\x02K\x0b\x86r\x8c\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x8d\x01\x00\x00tr\x8e\x01\x00\x00bhPh\x06h\x07K\x00\x85r\x8f\x01\x00\x00h\t\x87r\x90\x01\x00\x00Rr\x91\x01\x00\x00(K\x01K\x02K\x0b\x86r\x92\x01\x00\x00h)\x89C\xb0\xb5\xceK\xae\x80\xe1\x95=\xd8m\xabC\xa1t{=(\xc3:\x93x\x1d\xb4?\xf5\xdb8\x11\xb0\xbf\x08@\x04o{;\x0fE\xed?\xbb\xe8.4\x9f\x0c\r@\x95\x869M\xf7\xb9\xf8>\x14\xd3\xee\xfcyE\xb0=\xd8iQ\xab$\x0b\xb5=m\xb7ug\x8e\x84\xaf\xbd\x95]K\xe6\xf3*\x7f=7\x08\xd9\x0c\x89F\xa4?\x18Sq\xaa\x87\x8e\xdf?\x18Sq\xaa\x87\x8e\xdf?\xea\xf6{\xac\x86F\xa4?\xc4\x9f\x05\xc8\xf2\x08\x18@s\xec\xef\x8b|\xc1@>B\xf1\x12\xdb\xb7n\xe5?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x13\x8a\x8f\xfd\xf4\xcd\xc3>r\x93\x01\x00\x00tr\x94\x01\x00\x00bhWh\x06h\x07K\x00\x85r\x95\x01\x00\x00h\t\x87r\x96\x01\x00\x00Rr\x97\x01\x00\x00(K\x01K\x02K\x0b\x86r\x98\x01\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x99\x01\x00\x00tr\x9a\x01\x00\x00bh^h_h)C\x08\xdc\x84\x13\xafkr\x04@r\x9b\x01\x00\x00\x86r\x9c\x01\x00\x00Rr\x9d\x01\x00\x00hch_h)C\x08\xd8-\x839}_!@r\x9e\x01\x00\x00\x86r\x9f\x01\x00\x00Rr\xa0\x01\x00\x00hdheubh?)\x81r\xa1\x01\x00\x00}r\xa2\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85r\xa3\x01\x00\x00h\t\x87r\xa4\x01\x00\x00Rr\xa5\x01\x00\x00(K\x01K\x0b\x85r\xa6\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00\x14@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xa7\x01\x00\x00tr\xa8\x01\x00\x00bhIh\x06h\x07K\x00\x85r\xa9\x01\x00\x00h\t\x87r\xaa\x01\x00\x00Rr\xab\x01\x00\x00(K\x01K\x02K\x0b\x86r\xac\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xad\x01\x00\x00tr\xae\x01\x00\x00bhPh\x06h\x07K\x00\x85r\xaf\x01\x00\x00h\t\x87r\xb0\x01\x00\x00Rr\xb1\x01\x00\x00(K\x01K\x02K\x0b\x86r\xb2\x01\x00\x00h)\x89C\xb0G\xc8\xa6=\xd0\xd9\xaf\xbd\xa4_s\x97\x1b\x99\xb3=E\xdd\xe7\xc4\xab9\xac\xbdT\xfe\xa9\x92\xb2\xf4\x8a=J\xe4\xd3\x92\x0f\x0b\x13@O\x87\x9b\x87A\xbe\x16@R\xbe6\xd8%\x1e\x03?\xb2)*\xf7b\xb4\xa1=8\xe8)h\xc98\xae\xbd\xc89\xa9\x13\xf4\x86\x95=\x156\x91\x00\xd3B\xb1=\x1ce\x06#\xa6\x94s?>\xa9\xd0u"k\x15@\xf8]\xc9\xbb\x99\xac\xd3?\x1a\xcbj\xe2I\xd9!@\x07\x7fv\x83\xb9\xf5\xde?7Fqqj\xb0\x91?\xf4\xedv\x05E\x7f\xa3?\xe0\xe0[\n\x8a\x05\xaf>\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x14\x97Ce\xcd\x98\xc4>r\xb3\x01\x00\x00tr\xb4\x01\x00\x00bhWh\x06h\x07K\x00\x85r\xb5\x01\x00\x00h\t\x87r\xb6\x01\x00\x00Rr\xb7\x01\x00\x00(K\x01K\x02K\x0b\x86r\xb8\x01\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00"@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xb9\x01\x00\x00tr\xba\x01\x00\x00bh^h_h)C\x08L\xc2\x1f\xd1\xc9\x95\x04@r\xbb\x01\x00\x00\x86r\xbc\x01\x00\x00Rr\xbd\x01\x00\x00hch_h)C\x08I\x0b\xf6\x1b\x12\xdd@@r\xbe\x01\x00\x00\x86r\xbf\x01\x00\x00Rr\xc0\x01\x00\x00hdheubh?)\x81r\xc1\x01\x00\x00}r\xc2\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85r\xc3\x01\x00\x00h\t\x87r\xc4\x01\x00\x00Rr\xc5\x01\x00\x00(K\x01K\x0b\x85r\xc6\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xc7\x01\x00\x00tr\xc8\x01\x00\x00bhIh\x06h\x07K\x00\x85r\xc9\x01\x00\x00h\t\x87r\xca\x01\x00\x00Rr\xcb\x01\x00\x00(K\x01K\x02K\x0b\x86r\xcc\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xcd\x01\x00\x00tr\xce\x01\x00\x00bhPh\x06h\x07K\x00\x85r\xcf\x01\x00\x00h\t\x87r\xd0\x01\x00\x00Rr\xd1\x01\x00\x00(K\x01K\x02K\x0b\x86r\xd2\x01\x00\x00h)\x89C\xb0\xa1\xb9]\xd7\x0b)\xa0\xbd\xcf\xaa[\xc5a\x1d\x8a\xbd\xe6~8\x06\xdcT\xa5=\xb8T\xb3\x98)\x06\x10@\x14!U\x1cv\xfa\xb5?^\xba\xb3\xc9\x0c$\x11@\x94\xac\x93\xd7\xc8)\xf7>\xb9\x19\x17/\x99&\xa6=GZCy%\xc3\xa2=x\x9e\xeecc\x93\xa4=\xb4\xaa\x0c\x16\x1df\xa6\xbd\xad]tE%G0?\x0b\xc73\xdal\xaa\r@$\xa4\x89\r[6\x1a@\xe7FW\x1b$j\xe8?\xea\xb3\xa4a\xc2\x1dp?\xc4\xe2\x8b\xfc\x8b\x9e\xb1>j\xadd\xff`\x18\xad?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc4\xe2\x8b\xfc\x8b\x9e\xb1>\x19b\\\x9dH\xea\xc5>r\xd3\x01\x00\x00tr\xd4\x01\x00\x00bhWh\x06h\x07K\x00\x85r\xd5\x01\x00\x00h\t\x87r\xd6\x01\x00\x00Rr\xd7\x01\x00\x00(K\x01K\x02K\x0b\x86r\xd8\x01\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xd9\x01\x00\x00tr\xda\x01\x00\x00bh^h_h)C\x08\x1a\x8f\xe2&\xf3\xe4\x03@r\xdb\x01\x00\x00\x86r\xdc\x01\x00\x00Rr\xdd\x01\x00\x00hch_h)C\x08h\xb0\x10\xed\x98\xcd@@r\xde\x01\x00\x00\x86r\xdf\x01\x00\x00Rr\xe0\x01\x00\x00hdheubh?)\x81r\xe1\x01\x00\x00}r\xe2\x01\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85r\xe3\x01\x00\x00h\t\x87r\xe4\x01\x00\x00Rr\xe5\x01\x00\x00(K\x01K\x0b\x85r\xe6\x01\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00\x18@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xe7\x01\x00\x00tr\xe8\x01\x00\x00bhIh\x06h\x07K\x00\x85r\xe9\x01\x00\x00h\t\x87r\xea\x01\x00\x00Rr\xeb\x01\x00\x00(K\x01K\x02K\x0b\x86r\xec\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xed\x01\x00\x00tr\xee\x01\x00\x00bhPh\x06h\x07K\x00\x85r\xef\x01\x00\x00h\t\x87r\xf0\x01\x00\x00Rr\xf1\x01\x00\x00(K\x01K\x02K\x0b\x86r\xf2\x01\x00\x00h)\x89C\xb0\'$#\x97\xdc\n\xa5\xbd\xf3\xb46u\xd4\xe4\x94=3\xd2\x81\xe73Bv=?=\xc4>\xbb\xcao\xbd\xdde\x11\x00\xd5\xd3\x0b@\xc3N\xc2\xd2\xaap\x16@\x9d\x8f\x87\xfc\xb5\xcb\xf4>\xf4\xcb\xa2c\xcf\xe8\x9f=\xc4\xd2\xfe\\\x00\xec\xb3\xbd\x87"\x999C\x0b\xb0\xbd\xf1e\xc8\xc5\x11\xfb\xa2\xbd|\x8e\xedz\xbd\xbf\x14@\xcfZ(C\xcaz\xac?\xfe:\xef\xe86\xae\x1d@\xcfZ(C\xcaz\xac?/\x05\xb8MWw\xe3?\x8c\x14\xec\x82\x9d\x8fw>\xf1\xda\xab\x15\xe0\nh??Y\xf4\xc3@@\xac>\xec7!\xed\xd5 \xcc?\x00\x00\x00\x00\x00\x00\x00\x00\xaf\xb6\t\x1b>\xae\xc9>r\xf3\x01\x00\x00tr\xf4\x01\x00\x00bhWh\x06h\x07K\x00\x85r\xf5\x01\x00\x00h\t\x87r\xf6\x01\x00\x00Rr\xf7\x01\x00\x00(K\x01K\x02K\x0b\x86r\xf8\x01\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\xf9\x01\x00\x00tr\xfa\x01\x00\x00bh^h_h)C\x08Z\x99\xb1\x9ezW\x03@r\xfb\x01\x00\x00\x86r\xfc\x01\x00\x00Rr\xfd\x01\x00\x00hch_h)C\x08I\x0b\xf6\x1b\x12\xdd@@r\xfe\x01\x00\x00\x86r\xff\x01\x00\x00Rr\x00\x02\x00\x00hdheubh?)\x81r\x01\x02\x00\x00}r\x02\x02\x00\x00(h\x03K\x0bhBh\x06h\x07K\x00\x85r\x03\x02\x00\x00h\t\x87r\x04\x02\x00\x00Rr\x05\x02\x00\x00(K\x01K\x0b\x85r\x06\x02\x00\x00h\x10\x89CX\x00\x00\x00\x00\x00\x00\x1c@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x07\x02\x00\x00tr\x08\x02\x00\x00bhIh\x06h\x07K\x00\x85r\t\x02\x00\x00h\t\x87r\n\x02\x00\x00Rr\x0b\x02\x00\x00(K\x01K\x02K\x0b\x86r\x0c\x02\x00\x00h)\x89C\xb0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\r\x02\x00\x00tr\x0e\x02\x00\x00bhPh\x06h\x07K\x00\x85r\x0f\x02\x00\x00h\t\x87r\x10\x02\x00\x00Rr\x11\x02\x00\x00(K\x01K\x02K\x0b\x86r\x12\x02\x00\x00h)\x89C\xb0J7\x9c\xb3-\xcc\xa7=\xe1\xe6\xc0\xc5\xc4l\xb0=\xf3\x19\x9e\x11\xe5\xc4\x91\xbd\xfdS\xe4\x19\x8eb\xb4\xbd\xde\xd8\x9a1\xf6\xff\xef?\xa7\xba\x99\x8a\n5\x1d@\xa2\xe4\x92\xd2k\xa2\xbf>\xa0~\xf2\x98\xa2\x84\xaf\xbd\xa8\x003\x18\xb6\x1f\x88\xbd\xe6f3\xc8M\xc3\xb4=\xe4\xf7\x1c\xe8\xfe+\xb1\xbd^e\xce\t3e\xc5> }\x0e\xaeq\xf2\xaf?\xf5\x83F\x8aI\x93v?\x98\x1b $?\xc17?Q\x8d\x99\x1e5\x1a\x1a@\x00\x00\x00\x00\x00\x00\x00\x00\x15\xf4\xcd\xf1\x1a\xa7\xe6?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00u\x92\xc6/C[\x1e>r\x13\x02\x00\x00tr\x14\x02\x00\x00bhWh\x06h\x07K\x00\x85r\x15\x02\x00\x00h\t\x87r\x16\x02\x00\x00Rr\x17\x02\x00\x00(K\x01K\x02K\x0b\x86r\x18\x02\x00\x00h\x10\x89C\xb0\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 @\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00r\x19\x02\x00\x00tr\x1a\x02\x00\x00bh^h_h)C\x08\n\xe1\x8c8`\xed\x02@r\x1b\x02\x00\x00\x86r\x1c\x02\x00\x00Rr\x1d\x02\x00\x00hch_h)C\x08\xb0\x91\n\xdc\xe9\xbb@@r\x1e\x02\x00\x00\x86r\x1f\x02\x00\x00Rr \x02\x00\x00hdheubeub.'
#)
#pso = PSO(num_particles = 10, epoch = 2, continue_flag = True, pso = )

pso.run()

In [ ]:
n_folds = 5

base = [lasso, ENet, KRR, GBoost, svr, logistic]
meta = GBoost
meta_model = MetaModel(base_models = base, meta_model = meta)
score = cv_model(meta_model)